# Colab-VSGAN

My repo: [styler00dollar/VSGAN-tensorrt-docker](https://github.com/styler00dollar/VSGAN-tensorrt-docker/)

In [ ]:
!nvidia-smi

In [ ]:
!nvcc --version

In [ ]:
# check python version, for me its currently 3.7.12
!python --version

In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)



You need to get 2 files. Currently, the cuda version inside Colab is 11.1, that's why you need to get:
```
nv-tensorrt-repo-ubuntu1804-cuda11.4-trt8.2.3.0-ga-20220113_1-1_amd64.deb

(TensorRT 8.2 GA Update 2 for Ubuntu 18.04 and CUDA 11.0, 11.1, 11.2, 11.3, 11.4 and 11.5 DEB local repo Package)

and

TensorRT-8.2.3.0.Linux.x86_64-gnu.cuda-11.4.cudnn8.2.tar.gz/python/tensorrt-8.2.3.0-cp37-none-linux_x86_64.whl

(TensorRT 8.2 GA Update 2 for Linux x86_64 and CUDA 11.0, 11.1, 11.2, 11.3, 11.4 and 11.5 TAR Package)

```
You can download these files [here](https://developer.nvidia.com/nvidia-tensorrt-download). Warning: You need an account (which can be created for free). Put them into `gdrive/TensorRT`.

In [ ]:
#@title install tensorrt
import os
os.environ["os1"]="ubuntu1804"
os.environ["tag"]= "cuda11.4-trt8.2.3.0-ga-20220113" #@param
os.environ["version"]= "8.2.3-1+cuda11.4" #@param
data_path = '/content/drive/MyDrive/TensorRT/' #@param
os.chdir(data_path)
!sudo dpkg -i nv-tensorrt-repo-${os1}-${tag}_1-1_amd64.deb
!sudo apt-key add /var/nv-tensorrt-repo-${tag}/7fa2af80.pub
!sudo apt-get update
!sudo apt-get install libnvinfer8=${version} libnvonnxparsers8=${version} libnvparsers8=${version} libnvinfer-plugin8=${version} libnvinfer-dev=${version} libnvonnxparsers-dev=${version} libnvparsers-dev=${version} libnvinfer-plugin-dev=${version} python3-libnvinfer=${version}
!sudo apt-mark hold libnvinfer8 libnvonnxparsers8 libnvparsers8 libnvinfer-plugin8 libnvinfer-dev libnvonnxparsers-dev libnvparsers-dev libnvinfer-plugin-dev python3-libnvinfer
!sudo apt-get install tensorrt=8.2.3.0-1+cuda11.4
!sudo apt-get install python3-libnvinfer-dev=${version}

# install the whl with the matching python version
whl_path = "/content/drive/MyDrive/TensorRT/tensorrt-8.2.3.0-cp37-none-linux_x86_64.whl" #@param
!pip install {whl_path}

Install everything. Avoid compiling mmcv if you can, since the total install time will be long.

In [ ]:
#@title install (be patient, ~13min)
%cd /content/
!sudo rm -rf /workspace
!mkdir -p /workspace/tensorrt
%cd /workspace/tensorrt
!git clone https://github.com/styler00dollar/VSGAN-tensorrt-docker

# installing vapoursynth
!apt install ffmpeg autoconf libtool yasm python3.9 python3.9-venv python3.9-dev ffmsindex libffms2-4 libffms2-dev -y
!apt --fix-broken install
!git clone https://github.com/sekrit-twc/zimg.git && cd zimg && ./autogen.sh && ./configure && make -j4 && make install && cd .. && rm -rf zimg
!pip install Cython -U --force-reinstall
!wget https://github.com/vapoursynth/vapoursynth/archive/refs/tags/R57.zip -P /workspace/tensorrt
%cd /workspace/tensorrt/
!7z x /workspace/tensorrt/R57.zip
%cd /workspace/tensorrt/vapoursynth-R57
!./autogen.sh && ./configure && make && make install && cd .. && ldconfig
!ln -s /usr/local/lib/python3.7/site-packages/vapoursynth.so /usr/lib/python3.7/lib-dynload/vapoursynth.so
!pip install vapoursynth

# cupy / pycuda
!curl https://colab.chainer.org/install | sh -
!pip install pycuda

# pytorch tensorrt
!pip install https://github.com/NVIDIA/Torch-TensorRT/releases/download/v1.0.0/torch_tensorrt-1.0.0-cp37-cp37m-linux_x86_64.whl

# onnx
!pip install onnx onnxruntime onnxruntime-gpu

# installing onnx tensorrt with a workaround, error with import otherwise
%cd /workspace/tensorrt
!git clone https://github.com/onnx/onnx-tensorrt
%cd onnx-tensorrt
!python3 setup.py install

%cd /workspace
# downloading models
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/RealESRGANv2-animevideo-xsx2.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/RealESRGANv2-animevideo-xsx4.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/RealESRGAN_x4plus_anime_6B.pth
# fatal anime
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/4x_fatal_Anime_500000_G.pth
# rvp1
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/rvpV1_105661_G.pt
# sepconv
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/sepconv.pth
# EGVSR
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/EGVSR_iter420000.pth
# rife4 (fixed rife4.0 model)
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/rife40.pth
# RealBasicVSR_x4
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/RealBasicVSR_x4.pth
# cugan models
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/up2x-latest-conservative.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/up2x-latest-denoise1x.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/up2x-latest-denoise2x.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/up2x-latest-denoise3x.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/up2x-latest-no-denoise.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/up3x-latest-conservative.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/up3x-latest-denoise3x.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/up3x-latest-no-denoise.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/up4x-latest-conservative.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/up4x-latest-denoise3x.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/up4x-latest-no-denoise.pth
# film
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/FILM.tar.gz
!tar -zxvf FILM.tar.gz

# optional, rvp uses it to convert colorspace
!pip install kornia
# image read/write for image inference
!pip install opencv-python

# vs plugings from others
# https://github.com/HolyWu/vs-swinir
!pip install --upgrade vsswinir && python -m vsswinir

# pytorch hotfix
!pip uninstall torch -y
!pip uninstall torch -y
!pip3 install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio==0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

# onnx gpu hotfix
!pip install onnxruntime-gpu --force-reinstall -U

# jpg turbo
!sudo apt-get install -y libturbojpeg 
!pip install PyTurboJPEG

# render tools
!sudo apt install x264 -y

# downgrading tensorflow due to IndexError: list index out of range 
!pip install tensorflow==2.3.1 tensorflow-gpu==2.3.1

!pip install pytorch-msssim

%cd /workspace/tensorrt/VSGAN-tensorrt-docker

In [ ]:
#@title (optional) compile mmcv (basicvsrpp/RealBasicVSR)
#@markdown Compiling mmcv takes a while, only do this if you need it
# https://github.com/HolyWu/vs-basicvsrpp
!pip install --upgrade vsbasicvsrpp && python -m vsbasicvsrpp
# dependencies for RealBasicVSR_x4
# mmedit
!git clone https://github.com/open-mmlab/mmediting.git && cd mmediting && pip install -v -e .
# RealBasicVSR_x4 will download this
!wget "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" -P /root/.cache/torch/hub/checkpoints/

In [ ]:
#@title (optional) compile vs-mlrt
# updating cmake
%cd /content
!wget https://github.com/Kitware/CMake/releases/download/v3.23.0-rc1/cmake-3.23.0-rc1-linux-x86_64.sh
!chmod +x cmake-3.23.0-rc1-linux-x86_64.sh
!sh cmake-3.23.0-rc1-linux-x86_64.sh --skip-license
!cp /content/bin/cmake /usr/bin/cmake
!cp /content/bin/cmake /usr/lib/x86_64-linux-gnu/cmake
!cp /content/bin/cmake /usr/local/bin/cmake
!cp -r /content/share/cmake-3.23 /usr/local/share/
# upgrading g++
!sudo apt install build-essential manpages-dev software-properties-common -y
!sudo add-apt-repository ppa:ubuntu-toolchain-r/test -y
!sudo apt update -y && sudo apt install gcc-11 g++-11 -y
!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-11 11
!sudo update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-11 11
# getting vapoursynth sourcecode
%cd /content
!wget https://github.com/vapoursynth/vapoursynth/archive/refs/tags/R57.zip
!7z x R57.zip
# compiling
%cd /content/
!sudo rm -rf vs-mlrt
!git clone https://github.com/AmusementClub/vs-mlrt
%cd vs-mlrt/vstrt
!mkdir build
%cd build
!cmake .. -DVAPOURSYNTH_INCLUDE_DIRECTORY=/content/vapoursynth-R57/include
!make
!sudo make install
%cd /content
!wget https://github.com/AmusementClub/vs-mlrt/releases/download/v7/models.v7.7z
!7z x models.v7.7z

In [ ]:
#@title (optional) compile x265
!sudo apt-get install -y numactl
# updating cmake
%cd /content
!wget https://github.com/Kitware/CMake/releases/download/v3.23.0-rc1/cmake-3.23.0-rc1-linux-x86_64.sh
!chmod +x cmake-3.23.0-rc1-linux-x86_64.sh
!sh cmake-3.23.0-rc1-linux-x86_64.sh --skip-license
!cp /content/bin/cmake /usr/bin/cmake
!cp /content/bin/cmake /usr/lib/x86_64-linux-gnu/cmake
!cp /content/bin/cmake /usr/local/bin/cmake
!cp -r /content/share/cmake-3.23 /usr/local/share/
# upgrading g++
!sudo apt install build-essential manpages-dev software-properties-common -y
!sudo add-apt-repository ppa:ubuntu-toolchain-r/test -y
!sudo apt update -y && sudo apt install gcc-11 g++-11 -y
!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-11 11
!sudo update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-11 11
# compile x265
%cd /content
!sudo rm -rf x265
!git clone https://github.com/AmusementClub/x265
%cd x265/source/
!mkdir build
%cd build
!cmake .. -DNATIVE_BUILD=ON -DSTATIC_LINK_CRT=ON -DENABLE_AVISYNTH=OFF
!make 
!sudo make install
!cp /content/x265/source/build/x265 /usr/bin/x265 
!cp /content/x265/source/build/x265 /usr/local/bin/x265

`Runtime -> Restart runtime`

Dependencies are installed now.

# Render

In [ ]:
# if you want to use vs-mlrt, convert model into engine, you can find models in /content/models
# after creating the model, you can set the path in inference.py, be aware that cugan is not working
!/usr/src/tensorrt/bin/trtexec --fp16 --onnx=/content/models/waifu2x/upconv_7_anime_style_art_rgb/noise0_scale2.0x_model.onnx --minShapes=input:1x3x8x8 --optShapes=input:1x3x720x1280 --maxShapes=input:1x3x1080x1920 --saveEngine=/content/test.engine --tacticSources=+CUDNN,-CUBLAS,-CUBLAS_LT

In [ ]:
#@title inference.py
%%writefile /workspace/tensorrt/VSGAN-tensorrt-docker/inference.py
import sys
sys.path.append('/workspace/tensorrt/VSGAN-tensorrt-docker/')
import vapoursynth as vs
from src.esrgan import ESRGAN_inference # esrgan and realesrgan
from src.SRVGGNetCompact import SRVGGNetCompactRealESRGAN # realesrgan anime video
from src.vfi_model import video_model # any vfi model, in this case rvp1 as demonstration
from src.sepconv_enhanced import sepconv_model # uses cupy, no tensorrt
from src.rife import RIFE # tensorrt not possible
from vsswinir import SwinIR # https://github.com/HolyWu/vs-swinir # currently not tensorrt, didn't try
from src.egvsr import egvsr_model # currently not tensorrt
from src.cugan import cugan_inference
#from vsbasicvsrpp import BasicVSRPP
#from src.realbasicvsr import realbasicvsr_model
from src.film import FILM_inference

core = vs.core
vs_api_below4 = vs.__api_version__.api_major < 4
core = vs.core
core.num_threads = 16 # can influence ram usage
# only needed if you are inside docker
core.std.LoadPlugin(path='/usr/lib/x86_64-linux-gnu/libffms2.so')
core.std.LoadPlugin(path='/usr/local/lib/libvstrt.so')

# cfr video
clip = core.ffms2.Source(source='1080.mkv')
# vfr video (untested)
#clip = core.ffms2.Source(source='input.mkv', fpsnum = 24000, fpsden = 1001)
###############################################
# COLORSPACE
###############################################
# convert colorspace
clip = vs.core.resize.Bicubic(clip, format=vs.RGBS, matrix_in_s='709')
# convert colorspace + resizing
#clip = vs.core.resize.Bicubic(clip, width=848, height=480, format=vs.RGBS, matrix_in_s='709')

###############################################

# these demos work out of the box because docker also downloads the needed models, if you want other models, just add them
# you can combine everything however you want

###############################################
# MODELS (CUDA)
###############################################
# sepconv
#clip = sepconv_model(clip)
# RIFE4
#clip = RIFE(clip, multi = 2, scale = 1.0, fp16 = True, fastmode = False, ensemble = True, psnr_dedup = False, psnr_value = 70, ssim_dedup = True, ms_ssim_dedup = False, ssim_value = 0.999)
# VFI example for jit models
#clip = video_model(clip, fp16=False, model_path="/workspace/rvpV1_105661_G.pt")
# SwinIR
#clip = SwinIR(clip, task="lightweight_sr", scale=2)
# ESRGAN / RealESRGAN
#clip = ESRGAN_inference(clip=clip, model_path="/workspace/4x_fatal_Anime_500000_G.pth", tile_x=400, tile_y=400, tile_pad=10, fp16=False)
#clip = ESRGAN_inference(clip=clip, model_path="/workspace/RealESRGAN_x4plus_anime_6B.pth", tile_x=480, tile_y=480, tile_pad=16, fp16=False)
# RealESRGAN Anime Video example
# backends: tensorrt, cuda, onnx, quantized_onnx
#clip = SRVGGNetCompactRealESRGAN(clip, scale=2, fp16=True, backend_inference = "tensorrt")
# EGVSR
#clip = egvsr_model(clip, interval=15)
# BasicVSR++
# 0 = REDS, 1 = Vimeo-90K (BI), 2 = Vimeo-90K (BD), 3 = NTIRE 2021 - Track 1, 4 = NTIRE 2021 - Track 2, 5 = NTIRE 2021 - Track 3
#clip = BasicVSRPP(clip, model = 1, interval = 30, tile_x = 0, tile_y = 0, tile_pad = 16, device_type = 'cuda', device_index = 0, fp16 = False, cpu_cache = False)
# RealBasicVSR
#clip = realbasicvsr_model(clip, interval=15, fp16=True)
# cugan
# scales: 2 | 3 | 4, kind_model: no_denoise | denoise3x | conservative, backend_inference: cuda | onnx
# only cuda supports tiling
clip = cugan_inference(clip, fp16 = True, scale = 2, kind_model = "no_denoise", backend_inference = "cuda", tile_x=512, tile_y=512, tile_pad=10, pre_pad=0)
# FILM
# models: l1 | vgg | style
#clip = FILM_inference(clip, model_choise = "vgg")
# vs-mlrt
#clip = core.trt.Model(clip, engine_path="/content/test.engine", tilesize=[1280, 720], num_streams=15)

###############################################
# [NOT IN DOCKER] MODELS (NCNN)
# Only recommended for AMD GPUS, further instructions in README
###############################################
#from src.SRVGGNetCompact_ncnn import SRVGGNetCompactRealESRGAN_ncnn

# Rife ncnn
# 0 = rife-v3.1, 1 = rife-v3.0, 2 = rife-v2.4, 3 = rife-v2, 4 = rife-anime
#clip = core.rife.RIFE(clip, model=0, gpu_id=0, gpu_thread=2, tta=False, uhd=False, sc=False, list_gpu=False)
# RealESRGAN example
#clip = SRVGGNetCompactRealESRGAN_ncnn(clip, gpuid=0, model="models-DF2K", tta_mode=False, scale = 2, tilesize=0, param_path = None, bin_path = None)
# Waifu2x
# 0 = upconv_7_anime_style_art_rgb, 1 = upconv_7_photo, 2 = cunet (For 2D artwork. Slow, but better quality.)
#clip = core.w2xnvk.Waifu2x(clip, noise=0, scale=2, model=0, tile_size=0, gpu_id=0, gpu_thread=0, precision=16)

###############################################
# Deduplicated inference for faster inference
# only use this for upscaling
###############################################
#from src.dedup import return_frames
#frames_duplicated, frames_duplicating = return_frames(video_path, psnr_value=60)
#clip = core.std.DeleteFrames(clip, frames_duplicated)
# do upscaling here
#clip = core.std.DuplicateFrames(clip, frames_duplicating)
###############################################
# OUTPUT
###############################################
clip = vs.core.resize.Bicubic(clip, format=vs.YUV420P8, matrix_s="709")
clip.set_output()

In [ ]:
%cd /workspace/tensorrt/VSGAN-tensorrt-docker/
!sudo rm -rf /workspace/tensorrt/VSGAN-tensorrt-docker/example.mkv
!vspipe -c y4m inference.py - | ffmpeg -i pipe: /workspace/tensorrt/VSGAN-tensorrt-docker/example.mkv

In [ ]:
%cd /workspace/tensorrt/VSGAN-tensorrt-docker/
!sudo rm -rf /workspace/tensorrt/VSGAN-tensorrt-docker/example.mkv
!vspipe -c y4m inference.py - | x264 - --demuxer y4m -o /workspace/tensorrt/VSGAN-tensorrt-docker/example.mkv

In [ ]:
%cd /workspace/tensorrt/VSGAN-tensorrt-docker/
!sudo rm -rf /workspace/tensorrt/VSGAN-tensorrt-docker/example.mkv
!vspipe -c y4m inference.py - | x265 - --y4m --crf 23 -o /workspace/tensorrt/VSGAN-tensorrt-docker/example.mkv